In [26]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Загрузка данных

In [27]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [28]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [29]:
y = data['price']
X = data.drop(columns='price')

In [30]:
display(X.head())
display(X.info())

,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          7017 non-null   int64  
 1   condition     7017 non-null   int64  
 2   cylinders     7017 non-null   int64  
 3   odometer      7017 non-null   int64  
 4   title_status  7017 non-null   object 
 5   transmission  7017 non-null   object 
 6   drive         6626 non-null   object 
 7   size          5453 non-null   object 
 8   lat           7017 non-null   float64
 9   long          7017 non-null   float64
 10  weather       6837 non-null   float64
dtypes: float64(3), int64(4), object(4)
memory usage: 603.2+ KB


None

# Предобработка

In [31]:
lr = LinearRegression()
lr.fit(X,y)
# Мы получим ошибку с комментарием о том, 
# что не удалось превратить строковое значение в число с плавающей точкой (float).

ValueError: could not convert string to float: 'clean'

Для реализации данных методов мы использовали библиотеку category_encoders. Однако вы уже знаете и такой мощный инструмент, как scikit-learn. Данная библиотека содержит набор реализованных алгоритмов машинного обучения, метрик для оценки их качества, а также  класс preprocessing для предобработки данных, в частности — для кодирования категориальных признаков.

Представленная ниже таблица показывает соответствие типа кодирования классу в sklearn.processing.

Преимущество использования одной библиотеки состоит в типичности методов. Например, вы знаете, что для обучения модели в sklearn используется метод fit(). При кодировании признаков здесь также применяют fit() для подгонки кодировщика под выборку и transform() — для преобразования данных в числа.

_______________________
Из предыдущих модулей мы знаем, что при решении задач машинного обучения данные разбираются на обучающую (train) и валидационную (validation) выборки (последняя также может быть тестовой (test) выборкой). По аналогии подгонка кодировщика происходит на обучающей выборке, а трансформация — на обучающей и на тестовой.

Почему так? Потому что наша обученная модель не должна видеть данные, которые подаются в неё на тесте. Только так мы можем судить о том, что модель обучена качественно. То же самое и с кодировкой.

Давайте посмотрим на кодирование признака Образование способом «один-против-всех» (one vs all):

In [9]:
from sklearn.preprocessing  import LabelBinarizer
 
lb = LabelBinarizer()
 
education = ['нет', 'начальное', 'среднее', 'BSc', 'MSc', 'начальное', 'PhD']
lb.fit(education)
 
print('категории:', lb.classes_) 
lb.transform(['нет', 'MSc'])

категории: ['BSc' 'MSc' 'PhD' 'начальное' 'нет' 'среднее']


array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0]])

У класса LabelBinarizer, как и у двух остальных, есть атрибут classes_, который выводит список уникальных значений признака.

Вернёмся к нашей выборке. В ней присутствуют следующие категориальные признаки: condition, cylinders, title_status, transmission, drive, size.

При этом, признаки condition и cylinders — числовые, а title_status, transmission, drive, size — текстовые.

___________________________________

Важно производить кодирование номинальных признаков, даже если они уже представлены в числовом формате, так как, в отличие от порядковых признаков, категории номинальных являются независимыми. В случае порядкового кодирования таких признаков мы вносим искусственные закономерности в данные (например, чем больше числовой код цилиндров, тем лучше, хотя это необязательно так).

In [12]:
columns_to_change = ['cylinders', 'title_status', 'transmission', 'drive', 'size']
 
for column in columns_to_change:
 print('Число уникальных значений призкака {}: '.format(column), data[column].nunique())

Число уникальных значений призкака cylinders:  6
Число уникальных значений призкака title_status:  5
Число уникальных значений призкака transmission:  3
Число уникальных значений призкака drive:  3
Число уникальных значений призкака size:  4


Итак, нам подходит однократное кодирование. Применим его к выбранным столбцам. Так как у нас нет отдельной тестовой выборки, то мы используем только один метод — fit_transform(). В качестве аргумента передаём таблицу с выбранными для преобразования признаками.

С помощью метода get_feature_names_out() получим список новых названий колонок:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
 
one_hot_encoder = OneHotEncoder()
 
# 'учим' и сразу применяем преобразование к выборке, результат переводим в массив
data_onehot = one_hot_encoder.fit_transform(data[columns_to_change]).toarray() 
# запишем полученные названия новых колонок в отдельную переменную
column_names = one_hot_encoder.get_feature_names_out(columns_to_change)
print(column_names)

['cylinders_3' 'cylinders_4' 'cylinders_5' 'cylinders_6' 'cylinders_8'
 'cylinders_10' 'title_status_clean' 'title_status_lien'
 'title_status_missing' 'title_status_rebuilt' 'title_status_salvage'
 'transmission_automatic' 'transmission_manual' 'transmission_other'
 'drive_4wd' 'drive_fwd' 'drive_rwd' 'drive_nan' 'size_compact'
 'size_full-size' 'size_mid-size' 'size_sub-compact' 'size_nan']


Отлично, категориальные признаки закодированы. Теперь нам необходимо соединить преобразованные данные с исходными.

Напомним, что у библиотеки pandas есть дефолтный метод get_dummies() для получения однократного кодирования признаков. Однако OneHotEncoder способен принимать на вход как таблицы, так и numpy-массивы.

Задание 3.7

Какая форма (количество строк и столбцов) будет у нашего датасета о стоимости автомобилей после кодировки номинальных признаков однократным методом?

Для ответа на поставленный вопрос выполните следующие шаги:

1. Загрузите данные в локальную директорию с помощью команды !wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx.

2. Загрузите данные в память ноутбука с помощью функции read_excel() библиотеки pandas.

3. Закодируйте столбцы из списка columns_to_change.
4. Преобразуйте полученный массив закодированных данных в формат DataFrame, явно указав имена колонок.
5. Соедините новую таблицу с исходной.
6. Удалите закодированные столбцы columns_to_change из полученной таблицы.
7. Выведите на экран форму полученной таблицы.

In [33]:
# Создаем DataFrame из массива данных
df_onehot = pd.DataFrame(data_onehot, columns=column_names, index=data.index)

# Используйте метод pd.concat()
df_new = pd.concat([data, df_onehot], axis=1)

# ля удаления столбцов воспользуйтесь методом drop()
df_new = df_new.drop(columns=columns_to_change)

# Воспользуйтесь методом shape()
print("Размер итоговой таблицы:")
display(df_new.shape)

Размер итоговой таблицы:


(7017, 30)